In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import keras.backend as K

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import random
import seaborn as sns
import os
from PIL import Image
import cv2
tf.compat.v1.disable_eager_execution()


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_dir='../input/dogs-cats-images/dataset/training_set'
test_dir='../input/dogs-cats-images/dataset/test_set'

In [ ]:
train_cats=os.listdir(train_dir+'/'+'cats')
train_dogs=os.listdir(train_dir+'/'+'dogs')
test_cats=os.listdir(test_dir+'/'+'cats')
test_dogs=os.listdir(test_dir+'/'+'dogs')

In [ ]:
print('The Test Data is {}% of the Train Data'.format((len(test_cats+test_dogs)/len(train_cats+train_dogs))*100))

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.bar(x=['Cat','Dog'],height=[len(train_cats),len(train_dogs)],color=['gray','k'])
ax1.set_title('Train Class Balance')
ax2.bar(x=['Cat','Dog'],height=[len(test_cats),len(test_dogs)],color=['gray','k'])
ax2.set_title('Test Class Balance')


In [ ]:
def show_sample_images(k):
    plt.figure()
    cat_imgs=random.choices(train_cats,k=k)
    dog_imgs=random.choices(train_dogs,k=k)
    fig,ax=plt.subplots(2,k,figsize=(15,10))
    for i in range(k):
        img_cat=Image.open(train_dir+'/cats/'+cat_imgs[i])
        img_dog=Image.open(train_dir+'/dogs/'+dog_imgs[i])
        ax[0,i].imshow(img_cat)
        ax[1,i].imshow(img_dog)
    plt.show()

In [ ]:
show_sample_images(3)

In [ ]:
dog_img=random.choice(train_dogs)
dog_img=np.array(Image.open(train_dir+'/dogs/'+dog_img))
dog_img_scaled=np.array(dog_img)/255
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.imshow(dog_img)
ax1.set_title('Original Image')
ax2.imshow(dog_img_scaled)
ax2.set_title('Scaled Image')

In [ ]:
batch_size=32

In [ ]:
datagen=ImageDataGenerator(rescale=1/255)

In [ ]:
train_gen=datagen.flow_from_directory(train_dir,
                                      target_size=(150,150),
                                      batch_size=batch_size,
                                      class_mode='binary')
test_gen=datagen.flow_from_directory(test_dir,
                                      target_size=(150,150),
                                      batch_size=batch_size,
                                      class_mode='binary')

In [ ]:
def get_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape =(150,150,3)))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(rate = 0.3))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(rate = 0.2))
    model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(rate = 0.15))
    model.add(Flatten())
    model.add(Dense(units = 32, activation = 'relu'))
    model.add(Dropout(rate = 0.15))
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(Dropout(rate = 0.1))
    model.add(Dense(units = 2, activation = 'softmax'))
    return model

In [ ]:
base_model=get_model()
base_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
history=base_model.fit(train_gen,validation_data=test_gen,epochs=5,verbose=1)

In [ ]:
df=pd.DataFrame(history.history)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))

ax1.plot(df[['loss','val_loss']])
ax1.legend(['loss','val_loss'])
ax1.set_title('Loss per Epoch')

ax2.plot(df[['acc','val_acc']])
ax2.legend(['acc','val_acc'])
ax2.set_title('Accuracy per Epoch')

plt.show()

In [ ]:
base_model.summary()

In [ ]:
def get_outputs(model):
    layer_names=[]
    outputs=[]
    for layer in model.layers:
        if ('conv2d' in layer.name) or ('pooling' in layer.name) :
            layer_names.append(layer.name)
            outputs.append(layer.output)
    return layer_names,outputs

In [ ]:
layer_names,outputs=get_outputs(base_model)

In [ ]:
model=Model(base_model.input,outputs)

In [ ]:
def vis_activations(img,model,layer_names):
    activations=model.predict(img)
    images_per_row=16
    for layer_name,activation in zip(layer_names,activations):
        nb_features=activation.shape[-1]
        size=activation.shape[1]
        
        nb_cols=nb_features//images_per_row
        grid=np.zeros((size*nb_cols,size*images_per_row))
        
        for col in range(nb_cols):
            for row in range(images_per_row):
                feature_map=activation[0,:,:,col*images_per_row+row]
                feature_map-=feature_map.mean()
                feature_map/=feature_map.std()
                feature_map*=255
                feature_map=np.clip(feature_map,0,255).astype(np.uint8)
                grid[col*size:(col+1)*size, row*size:(row+1)*size] = feature_map
        scale = 1./size
        plt.figure(figsize=(scale*grid.shape[1], scale*grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(grid, aspect='auto', cmap='viridis')
    plt.show()
    return activations

In [ ]:
model.summary()

In [ ]:
img=train_gen.next()[0][0,:,:,:].reshape(-1,150,150,3)
activations=vis_activations(img,model,layer_names)

In [ ]:
base_model.summary()

In [ ]:
def get_Cam(img,label):
    predicted_output=base_model.output[:,label]
    last_conv_layer=base_model.get_layer(layer_names[-2])
    
    grads=K.gradients(predicted_output,last_conv_layer.output)[0]
    
    grads = K.mean(grads,axis=(0,1,2))
    evaluation_function=K.function([base_model.input],[grads,last_conv_layer.output[0]])
    
    grads_values,conv_output_values=evaluation_function(img)
    
    for i in range(126):
        conv_output_values[:,:,i]*=grads_values[i]
    
    
    heatmap=np.mean(conv_output_values,axis=-1)
    
    heatmap=np.maximum(heatmap,0)
    
    heatmap/=heatmap.max()
    
    return heatmap
    

In [ ]:
from skimage import data, color, io, img_as_float

In [ ]:
def show_random():
    b=test_gen.next()
    img=b[0][0,:,:,:]

    label=b[1][0]
    img=np.expand_dims(img,axis=0)
    pred_label = np.argmax(base_model.predict(img), axis=-1)[0]
    
    activations=model.predict(img)
    activation=activations[0][0,:,:,7]
    
    activation-=activation.mean()
    activation/=activation.std()
    activation*=255
    activation=np.clip(activation,0,255).astype(np.uint8)
    
    heatmap=get_Cam(img,pred_label)
    heatmap = cv2.resize(heatmap, (150, 150))
    heatmap = heatmap *255
    heatmap = np.clip(heatmap, 0, 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    img=img[0,:,:,:]
    sample_image_hsv = color.rgb2hsv(img)
    heatmap = color.rgb2hsv(heatmap)
    alpha=0.5
    sample_image_hsv[..., 0] = heatmap[..., 0]
    sample_image_hsv[..., 1] = heatmap[..., 1] * alpha

    img_masked = color.hsv2rgb(sample_image_hsv)
 
    super_imposed_image =heatmap*0.5+img
    super_imposed_image = np.clip(super_imposed_image, 0,255).astype(np.uint8)
    
    f,ax = plt.subplots(2,2, figsize=(15,8))
    ax[0,0].imshow(img)
    ax[0,0].set_title(f"True label: {label} \n Predicted label: {pred_label}")
    ax[0,0].axis('off')
    
    ax[0,1].imshow(activation)
    ax[0,1].set_title("Random feature map")
    ax[0,1].axis('off')
    
    ax[1,0].imshow(color.hsv2rgb(heatmap))
    ax[1,0].set_title("Class Activation Map")
    ax[1,0].axis('off')
    
    ax[1,1].imshow(img_masked)
    ax[1,1].set_title("Activation map superimposed")
    ax[1,1].axis('off')
    
    plt.show()
    
    
    

In [ ]:
show_random()